In [31]:
#Import Packages
using Pkg
using DataFrames
using CSV
using Plots
using GLM
using StatsBase
using Lathe
using MLBase
using ClassImbalance
using ROCAnalysis

In [27]:
# Read the file using CSV.File and convert it to DataFrame
df = DataFrame(CSV.File("books.csv"))
first(df,5)

┌ Warning: thread = 1 warning: parsed expected 12 columns, but didn't reach end of line around data row: 3350. Ignoring any extra columns on this row
└ @ CSV C:\Users\user\.julia\packages\CSV\Zl2ww\src\file.jl:613
┌ Warning: thread = 1 warning: parsed expected 12 columns, but didn't reach end of line around data row: 4704. Ignoring any extra columns on this row
└ @ CSV C:\Users\user\.julia\packages\CSV\Zl2ww\src\file.jl:613
┌ Warning: thread = 1 warning: parsed expected 12 columns, but didn't reach end of line around data row: 5879. Ignoring any extra columns on this row
└ @ CSV C:\Users\user\.julia\packages\CSV\Zl2ww\src\file.jl:613
┌ Warning: thread = 1 warning: parsed expected 12 columns, but didn't reach end of line around data row: 8981. Ignoring any extra columns on this row
└ @ CSV C:\Users\user\.julia\packages\CSV\Zl2ww\src\file.jl:613


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
,Int64,String,String,String,String,String,String,String,Int64,Int64,String,String
1,1,Harry Potter and the Half-Blood Prince (Harry Potter #6),J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
2,2,Harry Potter and the Order of the Phoenix (Harry Potter #5),J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
3,4,Harry Potter and the Chamber of Secrets (Harry Potter #2),J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
4,5,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
5,8,Harry Potter Boxed Set Books 1-5 (Harry Potter #1-5),J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [32]:
# Summary of dataframe
println(size(df))

(11127, 12)


In [23]:
describe(df)
#> (10000, 14)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,bookID,21310.9,1,20287.0,45641,0,Int64
2,title,,said the shotgun to the head.,,魔戒首部曲：魔戒現身,0,String
3,authors,,A.B. Yehoshua/Hillel Halkin,,Éric-Emmanuel Schmitt,0,String
4,average_rating,,Jr./Sam B. Warner,,5.00,0,String
5,isbn,,0.00,,9998691567,0,String
6,isbn13,,0008987059752,,9790007672386,0,String
7,language_code,,9780674842113,,zho,0,String
8,num_pages,,0,,eng,0,String
9,ratings_count,17936.5,0,745.0,4597666,0,Int64


In [33]:
names(df)

12-element Vector{String}:
 "bookID"
 "title"
 "authors"
 "average_rating"
 "isbn"
 "isbn13"
 "language_code"
 "  num_pages"
 "ratings_count"
 "text_reviews_count"
 "publication_date"
 "publisher"

In [34]:
countmap(df.Exited)

LoadError: ArgumentError: column name :Exited not found in the data frame

In [35]:
Lathe.preprocess.OneHotEncode(df,:publisher)
Lathe.preprocess.OneHotEncode(df,:publication_date)
select!(df, Not([:RowNumber, :CustomerId,:Surname,:Geography,:Gender,:Male]))
first(df,5)

LoadError: UndefVarError: OneHotEncode not defined

In [36]:
using Lathe.preprocess: TrainTestSplit
train, test = TrainTestSplit(df,.75);

In [37]:
fm = @formula(Exited ~ CreditScore + Age + Tenure + Balance + NumOfProducts + HasCrCard + IsActiveMember + EstimatedSalary + Female + France + Spain)
logit = glm(fm, train, Binomial(), ProbitLink())

LoadError: type NamedTuple has no field Exited

In [38]:
prediction = predict(logit,test)

LoadError: UndefVarError: logit not defined

In [39]:
# Convert probability score to class
prediction_class = [if x < 0.5 0 else 1 end for x in prediction];

prediction_df = DataFrame(y_actual = test.Exited, y_predicted = prediction_class, prob_predicted = prediction);
prediction_df.correctly_classified = prediction_df.y_actual .== prediction_df.y_predicted

LoadError: UndefVarError: prediction not defined

In [40]:
accuracy = mean(prediction_df.correctly_classified)

LoadError: UndefVarError: prediction_df not defined

In [41]:
# confusion_matrix = confusmat(2,prediction_df.y_actual, prediction_df.y_predicted)
confusion_matrix = MLBase.roc(prediction_df.y_actual, prediction_df.y_predicted)

LoadError: UndefVarError: prediction_df not defined

In [ ]:
# Import sklearn.metrics to julia
using PyCall
sklearn = pyimport("sklearn.metrics")